In [1]:
# Imports
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv('./data/processed_data.csv')
df.shape


In [ ]:
# Final labels
labels = ['patientState_good', 'patientState_caution', 'patientState_danger']
# Section labels
section_labels = {
    'fever': ['feverState_good', 'feverState_caution', 'feverState_danger'],
    'medication': ['medicationState_good', 'medicationState_caution', 'medicationState_danger'],
    'hydration': ['hydrationState_good', 'hydrationState_caution', 'hydrationState_danger'],
    'respiration': ['respirationState_good', 'respirationState_caution', 'respirationState_danger'],
    'skin': ['skinState_good', 'skinState_caution', 'skinState_danger'],
    'pulse': ['pulseState_good', 'pulseState_caution', 'pulseState_danger'],
    'general': ['generalState_good', 'generalState_caution', 'generalState_danger'],
}

section_features = {
    'fever': ['ageInMonths', 'temperature', 'feverDuration'],
    'medication': [
        'antibiotics',
        'antibioticsHowMany',
        'antibioticsHowMuch',
        'antipyretic',
        'antipyreticHowMany',
        'antipyreticHowMuch'],
    'hydration': [
        'crying',
        'diarrhea',
        'drinking',
        'lastUrination',
        'skinTurgor',
        'tearsWhenCrying',
        'tongue',
        'vomit'],
    'respiration': [
        'dyspnea',
        'respiratoryRate',
        'wheezing',
        'ageInMonths'],
    'skin': ['glassTest', 'rash', 'skinColor'],
    'pulse': ['pulse', 'ageInMonths'],
    'general': [
        'awareness',
        'bulgingFontanelleMax18MOld',
        'exoticTrip',
        'lastTimeEating',
        'pain',
        'painfulUrination',
        'seizure',
        'smellyUrine',
        'vaccinationIn14days',
        'vaccinationHowManyHoursAgo',
        'vaccinationWhat',
        'wryNeck'],
}


In [ ]:
# Separating to testing and training

# Prepare the data: You should split your data into training and test sets.
# The training set will be used to train the model and the test set will be used
# to evaluate the model's performance.
def split(dataset, ground_truth, test_size=0.2, val_size=0.2):
    _x, x_test, _y, y_test = train_test_split(
        dataset.to_numpy(),
        ground_truth.to_numpy(),
        test_size=test_size,
        random_state=42
    )
    x_train, x_val, y_train, y_val = train_test_split(
        _x,
        _y,
        test_size=val_size / 0.8,
        random_state=42
    )
    return x_train, y_train, x_val, y_val, x_test, y_test


In [ ]:
from tensorflow import keras
# Define the model: TensorFlow provides a high-level API for building and
# training neural network models. You should choose the appropriate model
# architecture for your problem and specify the hyperparameters,
# such as the number of hidden layers and the number of neurons in each layer.

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='./section_checkpoints/model_{epoch}',
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='./logs')
]


def build_model(hp):
    model = keras.Sequential([
        keras.layers.Dense(
            units=hp.Int('units', min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation', values=['tanh', 'sigmoid', 'relu']),
            input_shape=(45,)),
    ])
    layey_two = hp.Boolean('2nd_layer')
    with hp.conditional_scope('2nd_layer', True):
        if (layey_two):
            model.add(
                keras.layers.Dense(
                    units=hp.Int('units2', min_value=32, max_value=512, step=32),
                    activation=hp.Choice('activation', values=['tanh', 'sigmoid', 'relu']))
            )
    model.add(
        keras.layers.Dense(3, activation='softmax')
    )
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        # optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    model.build()
    return model


In [ ]:
# Compile the model: You should compile the model by specifying the optimizer,
# loss function, and evaluation metrics.
import keras_tuner

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

def get_tuner(section):
    return keras_tuner.tuners.Hyperband(
        build_model,
        objective='val_loss',
        # max_trials=50,
        executions_per_trial=2,
        overwrite=False,
        directory='section_tuner',
        project_name=section)
    

def tune(x,y,x_val,y_val,tuner):
    tuner.search(
        x,
        y,
        validation_data=(x_val, y_val),
        callbacks=[stop_early]
    )